In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Declare the selected keyword
selected_keyword = ["well-being"]  # Initial value
translations = ["hyvinvointi", "vointi"]
translations = translations + selected_keyword

# Load the CSV file into a DataFrame
df = pd.read_csv('data/parsed_data_2.csv')

# Filter out only threads
threads_df = df[df['msg_type'] == 'thread_start']

# Function to check if the keyword is in the text
def contains_keyword(text):
    tokens = word_tokenize(text.lower())  # Convert text to lowercase for case-insensitive matching
    return any(translation in tokens for translation in translations)

# Filter threads containing the keyword in title or thread_text
filtered_threads = threads_df[threads_df['title'].apply(contains_keyword) | threads_df['thread_text'].apply(contains_keyword)]

# Save the filtered results to a new CSV file
filtered_threads[['title', 'thread_text', 'datetime']].to_csv('data/filtered_threads.csv', index=False)
